*과소 적합*

테스트 세트의 성능이 향상될 여지가 아직 있을 때 일어난다. 발생하는 원인은 모델이 너무 단순하거나, 규제가 너무 많거나, 그냥 단순히 충분히 오래 훈련하지 않는 경우이다. 즉 네트워크가 훈련 세트에서 적절한 패턴을 학습하지 못했다는 뜻이다.

모델을 너무 오래 훈련하면 과대 적합되기 시작하고 테스트 세트에서 일반화 되지 못하는 패턴을 훈련 세트에서 학습한다. 과대적합과 과소적합 사이에서 균형을 잡아야 한다. 이를 위해 적절한 에포크 횟수동안 모델을 훈련하는 방법을 배워보자

과대 적합을 막는 가자 ㅇ좋은 방법은 더 많은 훈련 데이터를 사용하는 것이다. 많은 데이터에서 훈련한 모델은 자연적으로 일반화 성능이 더 좋다. 데이터를 더 준비할 수 없ㅇ르 때 그 다음으로 가장 좋은 방법은 규제와 같은 기법을 사용하는 것이다. 모델이 저장할 수 있는 정보의 양과 종류에 제약을 부과하는 방법이다. 네트워크가 소수의 패턴만 기억할 수 있다면 최적화 과정 동안 일반화 가능성이 가장 높은 중요한 패턴에 촛점을 맞출 것이다.

규제의 방법 중 *가중치 규제*와 *드롭아웃*을 알아보자

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

멀티-핫인코딩

정수 시퀀스를 0과 1로 이루어진 벡터로 변환한다. 정확하게 말하면 시퀀스[3,5]를 인덱스 3과 5만 1이고 나머지는 모두 0인 10,000차원 벡터로 변환한다는 말이다.

In [ ]:
NUM_WORDS = 1000

(train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(num_words=NUM_WORDS)

def multi_hot_sequences(sequences, dimension):
    # 0으로 채워진 (len(sequences), dimension) 크기의 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0  # results[i]의 특정 인덱스만 1로 설정합니다
    return results


train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)

In [ ]:
plt.plot(train_data[0])

# 과대 적합 예제

과대 적합을 막는 가장 간단한 방법은 모델의 규모를 축소하는 것이다. 즉, 모델에 있는 학습 가능한 파라미터의 수를 줄이는 것이다. (모델의 파라미터는 층의 개수와 층의 유닛 개수에 의해 결정된다.) 딥러닝에서는 모델의 학습 가능한 파라미어틔 수를 종종 모델의 용량이라고 말한다. 직관적으로 생각해보면 많은 파라미터를 가진 모델이 더 많은 "기억 용량"을 가진다. 이런 모델은 훈련 샘플과 타깃 사이를 일반화 능력이 없는 딕셔너리와 같은 매핑으로 완벽하게 학습할 수 있다. 하지만 이전에본 적 없는 데이터에서 예측을 할 땐 쓸모가 없을 것이다.

항상 기억해야할 점은 딥러닝 모델이 훈련 세트에서는 학습이 잘 되는 경향이 있지만, 진짜 해결할 문제는 학습이 아니라 **일반화**이다.

반면에 네트워크의 기억용량이 부족하다면 이러한 매핑을 쉽게 학습할 수 없을 것이다. 손실을 최소화하기 위해서는 예측 성능이 더 많은 압축된 표현을 학습해야 한다. 또한 너무 작은 모델을 만들면 훈련 데이터를 학습하기 어려울 것이다. "너무 많은 용량" 과 "충분하지 않은 용량"사이에서 균형을 잡아야 한다.

안타깝지만 모델의 적절한 크기나 구조를 결정하는 마법같은 공식은 없다. 

알맞은 모델의 크기를 찾으려면 비교적 적은 수의 층과 파라미터로 시작해서 ㄱ머증 손실이 감소할 때까지 새로운 층을 추가하거나 층의 크기를 늘리는 것이 좋다. 

## 기준 모델 만들기


In [ ]:
baseline_model = keras.Sequential([
    # `.summary` 메서드 때문에 `input_shape`가 필요합니다
    keras.layers.Dense(16, activation='relu', input_shape=(NUM_WORDS,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

baseline_model.summary()

In [ ]:
baseline_history = baseline_model.fit(train_data,
                                      train_labels,
                                      epochs=20,
                                      batch_size=512,
                                      validation_data=(test_data, test_labels),
                                      verbose=2)

Keras의 Model 이라는 패키지에서 제공하는 fit() 함수의 argument 중에 verbose가 있다.



verbose: Integer. 0, 1, or 2. 
Verbosity mode. 


0 = silent, 
1 = progress bar, 
2 = one line per epoch.



[출처] https://keras.io/models/model/

## 작은 모델 만들기

앞서 만든 기준 모델과 비교하기 위해 적은 수의 은닉 유닛을 가진 모델을 만들어보자.


In [ ]:
smaller_model = keras.Sequential([
    keras.layers.Dense(4, activation='relu', input_shape=(NUM_WORDS,)),
    keras.layers.Dense(4, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

smaller_model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy', 'binary_crossentropy'])

smaller_model.summary()

In [ ]:
smaller_history = smaller_model.fit(train_data,
                                    train_labels,
                                    epochs=20,
                                    batch_size=512,
                                    validation_data=(test_data, test_labels),
                                    verbose=2)

## 큰 모델 만들기

아주 큰 모델을 만들어 얼마나 빠르게 과대 적합이 시작되는지 볼 수 있다.

In [ ]:
bigger_model = keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(NUM_WORDS,)),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

bigger_model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy','binary_crossentropy'])

bigger_model.summary()

In [ ]:
bigger_history = bigger_model.fit(train_data, train_labels,
                                  epochs=20,
                                  batch_size=512,
                                  validation_data=(test_data, test_labels),
                                  verbose=2)

## 훈련 손실과 검증 손실 그래프 그리기


In [ ]:
def plot_history(histories, key='binary_crossentropy'):
  plt.figure(figsize=(16,10))

  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

  plt.xlabel('Epochs')
  plt.ylabel(key.replace('_',' ').title())
  plt.legend()

  plt.xlim([0,max(history.epoch)])


plot_history([('baseline', baseline_history),
              ('smaller', smaller_history),
              ('bigger', bigger_history)])

# 과대적합을 방지하기 위한 전략

## 가중치를 규제하기

"간단한 모델"이란 모델 파라미터의 분포로 봤을 때 엔트로피가 작은 모델이다. 따라서 과대적합을 완화시키는 일반적인 방법은 가중치가 작은 값을 가지도록 네트워크의 복잡도에 제약을 가하는 것이다. 이는 가중치 값의 분포를 좀 더 균일하게 만들어준다. 이를 "가중치 규제"라고 부른다. 네트워크의 손실 함수에 큰 가중치에 해당하는 비용을 추가한다. 이 비용은 두가지 형태가 있다.

> L1 규제는 가중치의 절댓값에 비례하는 비용이 추가된다.

> L2 규제는 가중치의 제곱에 비례하는 비용이 추가된다. 신경망에서는 L2 규제를 가중치 감쇠라고도 부른다.

L1 규제는 일부 가중치 파라미터를 0으로 만든다. L2 규제는 가중치 파라미터를 제한하지만 완전치 0으로 만들진 않는다. 이것이 L2 규제를 더 많이 사용하는 이유 중 하나다. 



In [ ]:
l2_model = keras.models.Sequential([
    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
                       activation='relu', input_shape=(NUM_WORDS,)),
    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
                       activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

l2_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

l2_model_history = l2_model.fit(train_data, train_labels,
                                epochs=20,
                                batch_size=512,
                                validation_data=(test_data, test_labels),
                                verbose=2)

l2(0.001)는 네트워크의 전체 손실에 층에 있는 가중치 행렬의 모든 값이 0.001 * weight_coefficient_value ** 2 만큼 더해진다는 의미이다. 이런 패널티는 훈련할 때만 추가된다. 따라서 테스트 단계보다 훈련 단계에서 네트워크 손실이 훨씬 더 크다.

In [ ]:
plot_history([('baseline', baseline_history),
              ('l2', l2_model_history)])

## 드롭아웃 추가하기

드롭아웃은 신경망에서 가장 효과적이고 널리 사용되는 규제 기법 중 하나이다. 

In [ ]:
dpt_model = keras.models.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(NUM_WORDS,)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

dpt_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy','binary_crossentropy'])

dpt_model_history = dpt_model.fit(train_data, train_labels,
                                  epochs=20,
                                  batch_size=512,
                                  validation_data=(test_data, test_labels),
                                  verbose=2)

In [ ]:
plot_history([('baseline', baseline_history),
              ('dropout', dpt_model_history)])

신경망 과대적합을 방지하기 위해 사용하는 방법

> 더 많은 훈련 데이터를 모은다.

> 네트워크의 용량을 줄인다.

> 가중치 규제를 추가한다.

> 드롭 아웃을 추가한다.